<a href="https://colab.research.google.com/github/eldercamposds/LangChain/blob/teste/ChatPython.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/LangChain
!pip install -r requirements.txt

# Setup

In [2]:
import numpy as np
from pinecone import Pinecone
import yaml
import os

In [3]:
with open('/content/drive/MyDrive/Colab Notebooks/LangChain/config.yaml', 'r') as config_file:
  config = yaml.safe_load(config_file)

os.environ['PINECONE_API_KEY'] = config['PINECONE_API_KEY']

In [4]:
pinecone_client = Pinecone()

#Pinecone

In [5]:
indices = pinecone_client.list_indexes() #retorna lista de todos os index do pinecone
for index in indices:
  index_name = index['name']
  print(f"Index: {index_name}")
  print(pinecone_client.describe_index(index_name))

Index: nlp
{'dimension': 2048,
 'host': 'nlp-ptq58qc.svc.aped-4627-b74a.pinecone.io',
 'metric': 'cosine',
 'name': 'nlp',
 'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
 'status': {'ready': True, 'state': 'Ready'}}


In [6]:
indice_name = 'nlp'

In [7]:
vetores = [np.random.normal(0,1,2048).tolist() for _ in range(5)] # criando 5 listas com distribuição normal, média 0, desvio padrão 1 e 2048 valores
ids = ["a", "b", "c", "d", "e"] #identiifcadores para os vetores
indice = pinecone_client.Index(indice_name)
indice.upsert(vectors=list(zip(ids, vetores)))#upset cria ou atualiza o indice

{'upserted_count': 5}

In [ ]:
print(indice.fetch(ids=["c"]))
#0.409804225 primeiro valor do indice

In [9]:
response = indice.fetch(ids=["c"])
if 'vctors' in response and 'c' in response['vectors']:
  retorna_valor = response['vectors']['c']['values']
  atualiza_valor = [x + 1 for x in retorna_valor]
  indice.upset(vectors=[('c', atualiza_valor)])
  print(indice.fetch(ids=['c']))
else:
  print('Vetor não encontrado!')

Vetor não encontrado!


In [10]:
indice.delete(ids=['d','e'])

{}

In [ ]:
print(indice.fetch(ids=['d','e']))

In [13]:
print(indice.describe_index_stats()) #descrição dos indices

{'dimension': 2048,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 5},
                'namespace1': {'vector_count': 5},
                'namespace2': {'vector_count': 3}},
 'total_vector_count': 13}


In [14]:
indice.upsert(vectors=list(zip(ids, vetores)), namespace="namespace1") # se não for definido o namespace será criado um namespace com nome em branco
indice.upsert(vectors=list(zip(["x", "y", "z"],
                                [np.random.normal(0, 1, 2048).tolist() for _ in range(3)])),
                                 namespace="namespace2")# criando segundo namespace com 3 indices

{'upserted_count': 3}

In [ ]:
print(indice.fetch(ids=['a'], namespace="namespace1"))

In [ ]:
print(indice.fetch(ids=['x'], namespace="namespace2"))

In [17]:
indice.delete(ids=['x'], namespace="namespace2")
print(indice.fetch(ids=['x'], namespace="namespace2"))

{'namespace': 'namespace2', 'usage': {'read_units': 1}, 'vectors': {}}


In [18]:
query_vector = np.random.normal(0, 1, 2048).tolist()
print(indice.query(vector=query_vector, top_k=3, include_values=False))


{'matches': [{'id': 'b', 'score': 0.0206540599, 'values': []},
             {'id': 'c', 'score': 0.00436178409, 'values': []},
             {'id': 'a', 'score': -0.0109398179, 'values': []}],
 'namespace': '',
 'usage': {'read_units': 5}}
